# LangGraph Introduction: Building Intelligent Workflows with OpenAI

This notebook demonstrates the fundamentals of LangGraph, a powerful library for creating stateful, multi-step workflows with Large Language Models.

## What You'll Learn

1. Basic LangGraph concepts and setup  
2. Creating simple linear workflows  
3. Building conditional flows  
4. State management across nodes  
5. Integration with OpenAI's GPT models

## Setup and Installation

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
!pip install langgraph langchain-openai python-dotenv

In [9]:
import os
from dotenv import load_dotenv
from typing import Dict, Any, List, Annotated
from typing_extensions import TypedDict

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.graph.message import add_messages

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## Configure OpenAI API

Make sure you have your OpenAI API key set up in your `.env` file or as an environment variable.

In [5]:
# Initialize OpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Test the connection
try:
    test_response = llm.invoke("Say 'Hello, LangGraph!'")
    print(f"✅ OpenAI connection successful: {test_response.content}")
except Exception as e:
    print(f"❌ OpenAI connection failed: {e}")
    print("Please check your API key in the .env file")

✅ OpenAI connection successful: Hello, LangGraph!


## 1. Basic LangGraph Concepts

Let's start with the fundamental building blocks of LangGraph.

### State Definition

In LangGraph, state is passed between nodes. We define it using `TypedDict`.

In [7]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    current_step: str
    user_input: str
    analysis_result: str
    summary: str
    final_response: str

print("✅ State definition created!")

✅ State definition created!


### Node Functions

Each node in the graph is a function that takes the state and returns an updated state.

In [8]:
def analyze_input(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        "Analyze the following text and identify the main topic, sentiment, and key points:\n\n{text}\n\nProvide a structured analysis."
    )
    chain = prompt | llm
    result = chain.invoke({"text": state["user_input"]})
    return {
        "analysis_result": result.content,
        "current_step": "analysis_complete",
        "messages": [result]
    }

def summarize_analysis(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        "Create a concise summary of the following analysis:\n\n{analysis}\n\nSummary:"
    )
    chain = prompt | llm
    result = chain.invoke({"analysis": state["analysis_result"]})
    return {
        "summary": result.content,
        "current_step": "summary_complete",
        "messages": [result]
    }

def generate_response(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        "Based on the analysis and summary below, generate a helpful response to the user:\n\nAnalysis: {analysis}\n\nSummary: {summary}\n\nOriginal Input: {input}\n\nResponse:"
    )
    chain = prompt | llm
    result = chain.invoke({
        "analysis": state["analysis_result"],
        "summary": state["summary"],
        "input": state["user_input"]
    })
    return {
        "final_response": result.content,
        "current_step": "complete",
        "messages": [result]
    }

print("✅ Node functions defined!")

✅ Node functions defined!


## 2. Creating Your First LangGraph Workflow

Now let's create a simple linear workflow that processes input through three steps:
1. Analyze input
2. Summarize the analysis
3. Generate a final response


In [10]:
def create_basic_workflow():
    """Create a basic linear LangGraph workflow"""

    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("analyze", analyze_input)
    workflow.add_node("summarize", summarize_analysis)
    workflow.add_node("respond", generate_response)

    # Add edges (defines flow)
    workflow.add_edge(START, "analyze")
    workflow.add_edge("analyze", "summarize")
    workflow.add_edge("summarize", "respond")
    workflow.add_edge("respond", END)

    return workflow.compile()

basic_app = create_basic_workflow()
print("✅ Basic workflow compiled!")

✅ Basic workflow compiled!


## 3. Running the Workflow

Let's test our linear workflow with a sample user input.

In [11]:
user_input = "I'm really excited about learning LangGraph! It seems like a powerful tool for building complex AI workflows. I can see many potential applications in my work."

print("🚀 Running workflow...\n")

try:
    result = basic_app.invoke({"user_input": user_input})

    print("=" * 50)
    print("📊 ANALYSIS RESULT")
    print("=" * 50)
    print(result["analysis_result"])

    print("\n" + "=" * 50)
    print("📝 SUMMARY")
    print("=" * 50)
    print(result["summary"])

    print("\n" + "=" * 50)
    print("💬 FINAL RESPONSE")
    print("=" * 50)
    print(result["final_response"])

    print("\n✅ Workflow completed successfully!")

except Exception as e:
    print(f"❌ Error running workflow: {e}")


🚀 Running workflow...

📊 ANALYSIS RESULT
### Analysis of the Text

**Main Topic:**
- The text discusses the author's enthusiasm for learning about LangGraph, a tool for building AI workflows.

**Sentiment:**
- The sentiment is positive. The author expresses excitement and optimism about the potential of LangGraph.

**Key Points:**
1. **Excitement for Learning:** The author is eager to learn about LangGraph.
2. **Powerful Tool:** LangGraph is described as a powerful tool, indicating its capabilities and effectiveness.
3. **Applications in Work:** The author sees many potential applications for LangGraph in their professional context, suggesting its relevance and utility in their field.

📝 SUMMARY
The text conveys the author's positive sentiment and enthusiasm for learning about LangGraph, a powerful tool for building AI workflows. The author expresses eagerness to explore its capabilities and recognizes its potential applications in their professional context, highlighting its relevance

## 4. Building Conditional Workflows

LangGraph supports conditional logic to dynamically route the workflow based on decisions made at runtime.

In this section, we’ll:
- Classify the input (technical / creative / general)
- Route to the appropriate handler node

🧾 State Update

In [12]:
class ConditionalAgentState(TypedDict):
    messages: Annotated[list, add_messages]
    current_step: str
    user_input: str
    classification: str
    final_response: str

🔎 Classification Node

In [13]:
def classify_input(state: ConditionalAgentState) -> ConditionalAgentState:
    prompt = ChatPromptTemplate.from_template(
        "Classify the following text as either 'technical', 'creative', or 'general':\n\n{text}\n\nClassification (respond with just one word):"
    )
    chain = prompt | llm
    result = chain.invoke({"text": state["user_input"]})
    return {
        "classification": result.content.strip().lower(),
        "current_step": "classified",
        "messages": [result]
    }

🧠 Handler Nodes

In [14]:
def technical_handler(state: ConditionalAgentState) -> ConditionalAgentState:
    prompt = ChatPromptTemplate.from_template(
        "Provide a technical analysis of the following text, including implementation details and best practices:\n\n{text}"
    )
    chain = prompt | llm
    result = chain.invoke({"text": state["user_input"]})
    return {
        "final_response": result.content,
        "current_step": "technical_complete",
        "messages": [result]
    }

def creative_handler(state: ConditionalAgentState) -> ConditionalAgentState:
    prompt = ChatPromptTemplate.from_template(
        "Provide a creative interpretation of the following text, focusing on mood, theme, and literary style:\n\n{text}"
    )
    chain = prompt | llm
    result = chain.invoke({"text": state["user_input"]})
    return {
        "final_response": result.content,
        "current_step": "creative_complete",
        "messages": [result]
    }

def general_handler(state: ConditionalAgentState) -> ConditionalAgentState:
    prompt = ChatPromptTemplate.from_template(
        "Provide a general overview of the following text, summarizing the main ideas and relevance:\n\n{text}"
    )
    chain = prompt | llm
    result = chain.invoke({"text": state["user_input"]})
    return {
        "final_response": result.content,
        "current_step": "general_complete",
        "messages": [result]
    }

🧭 Decision Function

In [15]:
def decide_path(state: ConditionalAgentState) -> str:
    classification = state.get("classification", "").lower()
    if "technical" in classification:
        return "technical"
    elif "creative" in classification:
        return "creative"
    return "general"

🧩 Create Conditional Workflow

In [16]:
def create_conditional_workflow():
    workflow = StateGraph(ConditionalAgentState)

    workflow.add_node("classify", classify_input)
    workflow.add_node("technical", technical_handler)
    workflow.add_node("creative", creative_handler)
    workflow.add_node("general", general_handler)

    workflow.add_conditional_edges(
        "classify",
        decide_path,
        {
            "technical": "technical",
            "creative": "creative",
            "general": "general"
        }
    )

    workflow.add_edge(START, "classify")
    workflow.add_edge("technical", END)
    workflow.add_edge("creative", END)
    workflow.add_edge("general", END)

    return workflow.compile()

conditional_app = create_conditional_workflow()
print("✅ Conditional workflow created!")

✅ Conditional workflow created!


## 5. Testing the Conditional Workflow

We’ll now test our conditional LangGraph pipeline with different types of input:
- Technical
- Creative
- General


🧪 Test: Technical Input

In [17]:
user_input = "I'm building a REST API using FastAPI and PostgreSQL. How should I handle authentication and background task processing efficiently?"

print("🔧 Testing TECHNICAL input...\n")

try:
    result = conditional_app.invoke({"user_input": user_input})
    print(f"📊 Classification: {result['classification']}")
    print(f"🎯 Final Step: {result['current_step']}")
    print("\n💬 Final Response:\n")
    print(result["final_response"])
except Exception as e:
    print(f"❌ Error: {e}")


🔧 Testing TECHNICAL input...

📊 Classification: technical
🎯 Final Step: technical_complete

💬 Final Response:

Building a REST API using FastAPI and PostgreSQL involves several considerations, especially when it comes to authentication and background task processing. Below is a technical analysis of how to implement these features efficiently, along with best practices.

### 1. Authentication

#### Implementation Details

FastAPI provides several ways to handle authentication, with OAuth2 being one of the most common methods. Here’s a step-by-step guide to implementing OAuth2 with password flow:

1. **Install Required Packages**:
   ```bash
   pip install fastapi[all] psycopg2-binary python-jose[cryptography] passlib[bcrypt]
   ```

2. **Database Setup**:
   Ensure you have a users table in your PostgreSQL database with fields for username, hashed password, and any other relevant user information.

3. **Password Hashing**:
   Use `passlib` to hash passwords before storing them in the d

🎨 Test: Creative Input

In [18]:
user_input = "The wind whispered secrets through the trees as the moon cast shadows like forgotten dreams across the field."

print("\n🎨 Testing CREATIVE input...\n")

try:
    result = conditional_app.invoke({"user_input": user_input})
    print(f"📊 Classification: {result['classification']}")
    print(f"🎯 Final Step: {result['current_step']}")
    print("\n💬 Final Response:\n")
    print(result["final_response"])
except Exception as e:
    print(f"❌ Error: {e}")



🎨 Testing CREATIVE input...

📊 Classification: creative
🎯 Final Step: creative_complete

💬 Final Response:

In the hushed embrace of twilight, the wind became a soft-spoken confidant, weaving its way through the gnarled branches of ancient trees. Each rustle of leaves carried with it the weight of untold stories, secrets that danced on the edge of memory, elusive yet tantalizing. The moon, a silent sentinel in the night sky, draped the landscape in a silvery veil, its light spilling like liquid nostalgia across the field. Shadows stretched and curled, reminiscent of dreams long abandoned, flickering at the periphery of consciousness.

The atmosphere was thick with a sense of yearning, a bittersweet nostalgia that hung in the air like the scent of rain on dry earth. The trees stood as guardians of these whispered tales, their bark etched with the passage of time, while the moonlight painted the world in shades of longing and reflection. Each shadow seemed to pulse with life, a reminder

## 6. Conclusion and Next Steps

Congratulations! You've built both linear and conditional LangGraph workflows from scratch using OpenAI’s GPT models.

---

### What you learned:

- How to define and track state with `TypedDict`
- How to implement clean node functions that return only updated fields
- How to route execution flow using linear or conditional logic
- How to build production-ready LLM pipelines with LangGraph

---

### Useful Resources:

- 🔗 [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- 🔗 [LangChain Docs](https://docs.langchain.com/)
- 🔗 [OpenAI Python SDK](https://github.com/openai/openai-python)

---

### Read the full article on Medium:**
[LangGraph Introduction: Building Intelligent Workflows with OpenAI](https://medium.com/@your-medium-handle/langgraph-introduction-building-intelligent-workflows-with-openai-xxxxx)

🤝 **Thanks for following along — happy building with LangGraph!**